In [ ]:
import xarray as xr

In [ ]:
target_chunks = {'lat': 600, 'lon': 1440, 'time': 1}
# target_chunks = {'lat': 600, 'lon': 1440, 'time': 29}
target_chunks

In [ ]:
def format_function(time):
    return f"s3://nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn_{time}.nc"

In [ ]:
from pangeo_forge_recipes.patterns import FilePattern, ConcatDim, MergeDim
from pangeo_forge_recipes import patterns

years = list(range(1950, 2015))
time_dim = ConcatDim("time", keys=years)

In [ ]:
from pangeo_forge_recipes.patterns import FilePattern

pattern = FilePattern(format_function, time_dim, file_type="netcdf4")
pattern = pattern.prune(nkeep=2)
pattern

In [ ]:
import apache_beam as beam
from pangeo_forge_recipes.transforms import OpenURLWithFSSpec, OpenWithXarray, StoreToZarr

In [ ]:
import os

if target_chunks['time'] == 29:
    store_name = 'data-0'
elif target_chunks['time'] == 1:
    store_name = 'data-1'
target_root = f"s3://carbonplan-benchmarks/data/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn"
target_store = os.path.join(target_root, store_name)
target_store

In [ ]:
transforms = (
    beam.Create(pattern.items())
    | OpenURLWithFSSpec(open_kwargs={'anon': True})
    | OpenWithXarray(file_type=pattern.file_type)
    | StoreToZarr(
        store_name=store_name,
        target_root=target_root,
        combine_dims=pattern.combine_dim_keys,
        target_chunks=target_chunks,
    )
)

In [ ]:
with beam.Pipeline() as p:
    p | transforms

In [ ]:
ds = xr.open_dataset(target_store, engine="zarr")
ds